
```
conda create -n tf_env python=3.10 pip -y
conda activate tf_env
pip install tensorflow==2.17.1 keras==3.5.0 numpy==1.26.4 scipy==1.13.1 seaborn==0.13.2 tensorflow-model-optimization==0.8.0 scikit-plot==0.3.7 tf_keras==2.17.0
```

In [37]:
# ! pip install tensorflow-model-optimization

In [38]:
import tensorflow as tf
import numpy as np
import scipy

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")

TensorFlow version: 2.17.1
NumPy version: 1.26.4
SciPy version: 1.13.1


In [39]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow_model_optimization.python.core.keras.compat import keras

In [40]:
path = "/home/pedro/projetoDL/dataset/processado/"
log_path_tmp = "/home/pedro/projetoDL/log"
save_path = "/home/pedro/projetoDL/log/exp_20241103170505/3_fold/prunning/"

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Carregar os dados
Y = np.load(path + 'Y_train_NewApproach_Injected_v2.npz')
Y= Y.f.arr_0

X = np.load(path + 'X_train_NewApproach_Injected_v2.npz')
X = X.f.arr_0

In [41]:
import tensorflow_model_optimization as tfmot
from sklearn.model_selection import KFold
import numpy as np
import os

# Configurar o KFold e selecionar o fold 3
skf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 0

for train_index, val_index in skf.split(X, Y):
    fold_no += 1
    if fold_no == 3:  # Selecionar o fold 3
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
        break



# Verificar os dados
print(f"Tamanho de x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"Tamanho de x_val: {x_val.shape}, y_val: {y_val.shape}")
print("Distribuição em y_train:", np.unique(y_train, return_counts=True))
print("Distribuição em y_val:", np.unique(y_val, return_counts=True))

Tamanho de x_train: (514647, 44, 116), y_train: (514647,)
Tamanho de x_val: (128662, 44, 116), y_val: (128662,)
Distribuição em y_train: (array([0, 1]), array([134480, 380167]))
Distribuição em y_val: (array([0, 1]), array([33964, 94698]))


In [42]:
model_fold3_path = "/home/pedro/projetoDL/log/exp_20241103170505/3_fold/fold_n_3_2dconv_fixed.h5"

In [44]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# Configuração do pruning
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step for pruning
batch_size = 256
epochs = 2  # Ajuste conforme necessário
validation_split = 0.1  # 10% dos dados de treino serão usados como validação

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=end_step
    )
}

# Construir modelo Sequential explicitamente
model = Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=(44, 116, 1), kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (5, 5), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Carregar pesos do modelo salvo
model.load_weights(model_fold3_path)

# Criar um modelo podado a partir do modelo explícito
try:
    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # Compilar o modelo podado
    model_for_pruning.compile(
        optimizer=Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Treinar o modelo podado
    model_for_pruning.fit(
        x_train, y_train,
        validation_split=validation_split,
        batch_size=batch_size,
        epochs=epochs
    )

    # Avaliar o modelo podado
    scores = model_for_pruning.evaluate(x_val, y_val)
    print(f"Loss após pruning: {scores[0]}, Acurácia após pruning: {scores[1]}")

    # Salvar o modelo podado
    model_for_pruning.save(os.path.join(save_path, "pruned_fold3_model.h5"))

except ValueError as e:
    print(f"Erro ao aplicar pruning: {e}")
    print(f"Tipo do modelo: {type(model)}")

# Compilar o modelo podado
model_for_pruning.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo podado
model_for_pruning.fit(
    x_train, y_train,
    validation_split=validation_split,
    batch_size=batch_size,
    epochs=epochs
)

# Avaliar o modelo podado
scores = model_for_pruning.evaluate(x_val, y_val)
print(f"Loss após pruning: {scores[0]}, Acurácia após pruning: {scores[1]}")

# Salvar o modelo podado
model_for_pruning.save(os.path.join(save_path, "pruned_fold3_model.h5"))


Erro ao aplicar pruning: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: Sequential.
Tipo do modelo: <class 'keras.src.models.sequential.Sequential'>


NameError: name 'model_for_pruning' is not defined

In [ ]:
# Fazer previsões com o modelo podado
y_pred_scores = model_for_pruning.predict(x_val)  # Probabilidades
y_pred_class = (y_pred_scores > 0.5).astype(int)  # Classificação binária

# Salvar os resultados
np.savez_compressed(
    os.path.join(save_path, "fold_3_y_val_y_pred_class_y_pred_scores.npz"),
    y_val=y_val,
    y_pred_class=y_pred_class,
    y_pred_scores=y_pred_scores
)

# Salvar o modelo podado
model_for_pruning.save(os.path.join(save_path, "pruned_fold3_model.h5"))

# (Opcional) Converter para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_pruning)
tflite_model = converter.convert()

with open(os.path.join(save_path, "pruned_fold3_model.tflite"), "wb") as f:
    f.write(tflite_model)

print("Modelo podado e resultados salvos com sucesso!")

Tamanho de x_train: (514647, 44, 116), y_train: (514647,)
Tamanho de x_val: (128662, 44, 116), y_val: (128662,)
Distribuição em y_train: (array([0, 1]), array([134480, 380167]))
Distribuição em y_val: (array([0, 1]), array([33964, 94698]))
O modelo carregado é do tipo Sequential.
Erro ao aplicar pruning: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: Sequential.
Tipo do modelo: <class 'keras.src.models.sequential.Sequential'>


ValueError: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: Sequential.

# Evaluate prunned model

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
import scikitplot as skplt


import matplotlib.pyplot as plt
import sklearn

from scipy.stats import ks_2samp
from sklearn.metrics import roc_curve, auc


def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado 
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        skplt.metrics.plot_ks_statistic(y, y_pred_scores)
        plt.savefig("ks_plot.png")
        plt.show()
        y_pred_scores = y_pred_scores[:, 1]
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

In [ ]:
from keras.models import load_model
import time
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import scikitplot as skplt
import gc

# Caminho para os arquivos e modelo podado
path = "/home/pedro/projetoDL/dataset/processado/"
tmp_path = "/home/pedro/projetoDL/log/exp_20241103170505/3_fold/"
load_path = tmp_path
save_path = tmp_path + 'prunning/results_pruned/'

# Verifica se o diretório existe, se não, cria o diretório
if not os.path.exists(save_path):
    os.makedirs(save_path)

file1 = open(save_path + 'TESTE_model_pruned_log.txt', "a")

# Carregar os dados de teste
Y = np.load(path + 'Y_test_Driving_NewApproach_Injected_v2.npz').f.arr_0
X = np.load(path + 'X_test_Driving_NewApproach_Injected_v2.npz').f.arr_0

In [ ]:
from sklearn.model_selection import train_test_split

# Definir 2/3 dos dados
X_subset, _, Y_subset, _ = train_test_split(
    X, Y, train_size=1/3, stratify=Y, random_state=42  # Stratify para manter a proporção de classes
)

# Agora X_subset e Y_subset têm 2/3 dos dados originais
X = X_subset
Y = Y_subset

In [ ]:
# Carregar o modelo podado do fold 3
model = load_model(load_path + 'pruned_fold3_model.h5')

# Fazer previsões
start = time.time()
y_pred_scores = model.predict(X)
end = time.time()

# Tempo de execução
print(f"\nSubset size: {X.shape[0]}\n", file=file1)
print(f"Runtime of the program is {end - start}", file=file1)

total_time = end - start
timesample = total_time / y_pred_scores.shape[0]
print(f"us/sample is {timesample * 1000000}", file=file1)

# Gerar as classificações e probabilidades
y_pred_class = (y_pred_scores > 0.5).astype("int64")
y_pred_scores_0 = 1 - y_pred_scores
y_pred_scores = np.concatenate([y_pred_scores_0, y_pred_scores], axis=1)

# Calcular métricas de desempenho
accuracy, recall, precision, f1, auroc, aupr = compute_performance_metrics(Y, y_pred_class, y_pred_scores)

# Salvar resultados
print(f'Results for pruned model: Recall of {recall}; accuracy of {accuracy}; precision of {precision}; f1 of {f1}; auroc of {auroc}; aupr of {aupr}', file=file1)

# Plotar e salvar a matriz de confusão
sns.set(rc={'figure.figsize': (8, 8)})
subplot = skplt.metrics.plot_confusion_matrix(Y, y_pred_class, normalize=True)
subplot.set_ylim(-0.5, 1.5)
plt.savefig(save_path + "conf_matrix_pruned.png")
plt.show()

# Plotar e salvar a curva ROC
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y, y_pred_scores[:, 1])
auc_keras = auc(fpr_keras, tpr_keras)

sns.set(rc={'figure.figsize': (8, 8)})
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='black')
plt.plot(fpr_keras, tpr_keras, marker='.', label='Pruned Model (auc = %0.4f)' % auc_keras)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig(save_path + "plot_roc_pruned.png")
plt.show()

# Limpar recursos
file1.close()
del model
time.sleep(3)
gc.collect()

print("Avaliação do modelo podado concluída!")